# Introduction to FermiLib 0.1

## $-1.7 \, a^\dagger_3 a_1$ and $(1 + 2i) \, a^\dagger_4 a^\dagger_2 a_3 a_0$ are FermionTerms

In [ ]:
from fermion_operators import FermionTerm

term_1 = -1.7 * FermionTerm('3^ 1')
term_2 = (1. + 2.j) * FermionTerm('4^ 2^ 3 0')

print term_1
print term_2

## FermionTerms support many built-in operators

In [ ]:
print 2. * term_1
print term_1 / 2.

In [ ]:
print term_1 == term_2
print term_1 != term_2

In [ ]:
print abs(term_2)
print len(term_2)

In [ ]:
print term_1 ** 2
print term_1 * term_2

In [ ]:
for x in term_2:
    print x

In [ ]:
print (4, 1) in term_2
print (1, 0) in term_2

In [ ]:
print term_1
term_1 *= 3.
print term_1
term_1 /= 3.
print term_1

## FermionOperators store the sum of FermionTerms

In [ ]:
operator_1 = 8.2 * term_1 - 3.0 * term_2
operator_2 = term_2 ** 2 - operator_1
print operator_1
print operator_2

In [ ]:
print operator_1 - 2. * operator_2
print term_1 + operator_2 - term_2

In [ ]:
print operator_1
operator_1 -= 1000. * term_1 * term_2
print operator_1

In [ ]:
operator_2 **= 2
print operator_2

In [ ]:
print term_1 in operator_1
print term_1 in operator_2

In [ ]:
for term in operator_2:
    term *= 0.0001
print operator_2

In [ ]:
print operator_2[term_1.operators]
print operator_1[term_1.operators]

## Of course, there are many custom methods

In [ ]:
print term_1.commutator(term_2)
print term_1.commutator(term_1)

In [ ]:
print term_2
print term_2.hermitian_conjugated()

In [ ]:
term_3 = term_2 * term_2.hermitian_conjugated()
print term_3
print term_3.is_normal_ordered()
print term_3.normal_ordered()

In [ ]:
print term_1.is_molecular_term()
print term_2.is_molecular_term()

In [ ]:
print operator_1.list_terms()
print operator_1.list_coefficients()

## We can map to QubitTerms and QubitOperators (and back)

In [ ]:
from qubit_operators import QubitTerm
my_term = 77. * QubitTerm('X0 Y2 Z3')
print my_term

In [ ]:
print term_1.jordan_wigner_transform()
print term_1.bravyi_kitaev_transform()

In [ ]:
qubit_op_1 = term_1.jordan_wigner_transform()
qubit_op_2 = operator_1.jordan_wigner_transform()
print qubit_op_1
print qubit_op_2

In [ ]:
for x in qubit_op_1:
    print x
    print x.reverse_jordan_wigner()

In [ ]:
qubit_op_2 -= 2. * qubit_op_1 ** 2
print qubit_op_2

## QubitTerms and QubitOperators can map to SparseOperators

In [ ]:
sparse_op_1 = qubit_op_1.get_sparse_operator()
print sparse_op_1

In [ ]:
sparse_op_1 *= 2.
print sparse_op_1

In [ ]:
fermion_operator = term_1 + term_1.hermitian_conjugated()
print fermion_operator
qubit_term = fermion_operator.jordan_wigner_transform()
print qubit_term
sparse_operator = qubit_term.get_sparse_operator()
print sparse_operator.eigenspectrum()

In [ ]:
print sparse_operator.is_hermitian()
ground_state_energy, ground_state = sparse_operator.get_ground_state()
print ground_state_energy
print sparse_operator.expectation(ground_state)

## Lattice models

In [ ]:
from hubbard import fermi_hubbard
x_dimension = 2
y_dimension = 2
tunneling = 2.
coulomb = 1.
magnetic_field = 0.5
chemical_potential = 0.25
periodic = 1
spinless = 1

hubbard_model = fermi_hubbard(
    x_dimension, y_dimension, tunneling, coulomb,
    chemical_potential, magnetic_field, periodic, spinless)
print hubbard_model

In [ ]:
jw_hamiltonian = hubbard_model.jordan_wigner_transform()
bk_hamiltonian = hubbard_model.bravyi_kitaev_transform()
print jw_hamiltonian
print bk_hamiltonian

In [ ]:
jw_sparse = jw_hamiltonian.get_sparse_operator()
bk_sparse = bk_hamiltonian.get_sparse_operator()
print jw_sparse.eigenspectrum() - bk_sparse.eigenspectrum()

## Jellium

In [ ]:
from jellium import jellium_model

n_dimensions = 1
grid_length = 5
length_scale = 3.
spinless = 1

momentum_jellium = jellium_model(n_dimensions, grid_length, length_scale,
                                 spinless, momentum_space=True)
print momentum_jellium

In [ ]:
momentum_qubit = momentum_jellium.jordan_wigner_transform()
print momentum_qubit

In [ ]:
position_jellium = jellium_model(n_dimensions, grid_length, length_scale,
                                 spinless, momentum_space=False)
position_qubit = position_jellium.jordan_wigner_transform()
print position_jellium
print position_qubit

In [ ]:
momentum_sparse = momentum_qubit.get_sparse_operator()
position_sparse = position_qubit.get_sparse_operator()
print momentum_sparse.eigenspectrum() - position_sparse.eigenspectrum()

In [ ]:
n_dimensions = 1
length_scale = 2.
spinless = 1

for grid_length in range(3, 17):
    
    if not grid_length % 2:
        continue
        
    momentum_jellium = jellium_model(n_dimensions, grid_length, length_scale,
                                     spinless, momentum_space=True)
    momentum_qubit = momentum_jellium.jordan_wigner_transform()
    
    position_jellium = jellium_model(n_dimensions, grid_length, length_scale,
                                     spinless, momentum_space=False)
    position_qubit = position_jellium.jordan_wigner_transform()
    
    print 'At {} qubits...'.format(momentum_jellium.n_qubits())
    print 'Momentum space Hamiltonian has {} terms.'.format(len(momentum_qubit))
    print 'Position space Hamiltonian has {} terms.\n'.format(len(position_qubit))

## MolecularData class stores data about molecules

In [ ]:
from molecular_data import MolecularData

diatomic_bond_length = 8.
geometry = [('Na', (0., 0., 0.)),
            ('Cl', (0., 0., diatomic_bond_length))]
basis = '6-31g'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

molecule = MolecularData(
    geometry, basis, multiplicity, charge, description)

In [ ]:
print molecule.name
print molecule.n_atoms
print molecule.atoms
print molecule.protons

In [ ]:
from chemical_series import make_atom

atomic_symbols = ['C', 'Na', 'Ca', 'Te', 'Cu', 'Fe', 'Ag']

for symbol in atomic_symbols:
    atom = make_atom(symbol, basis)
    print symbol, atom.multiplicity

## Running Psi4 or Pyscf to populate MolecularData

In [ ]:
from run_psi4 import run_psi4
from run_pyscf import run_pyscf

# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
bond_length = 0.7414
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_fci = 1

# Perform calculation.
molecule = MolecularData(
        geometry, basis, multiplicity)
molecule = run_psi4(
    molecule, run_scf=run_scf, run_mp2=run_mp2, run_fci=run_fci)

print 'Hartree-Fock energy', molecule.hf_energy
print 'MP2 energy', molecule.mp2_energy
print 'FCI energy', molecule.fci_energy
print 'Nuclear repulsion energy', molecule.nuclear_repulsion

In [ ]:
bond_lengths = []
hf_energies = []
fci_energies = []
for point in range(2, 20):
    
    bond_length = 0.1 * float(point)
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    bond_lengths += [bond_length]
    
    molecule = MolecularData(
        geometry, basis, multiplicity, description=str(bond_length))
    molecule = run_pyscf(molecule, run_scf=run_scf, run_fci=run_fci)

    print '\nBond length in Bohr', bond_length
    print 'Hartree-Fock energy', molecule.hf_energy
    print 'FCI energy', molecule.fci_energy
    fci_energies += [molecule.fci_energy]
    hf_energies += [molecule.hf_energy]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(0)
plt.plot(bond_lengths, fci_energies, 'x-')
plt.plot(bond_lengths, hf_energies, 'o-')
plt.ylabel('Energy in Hartree')
plt.xlabel('Bond length in Bohr')
plt.show()

## $\sum_{pq} h_{pq}\, a^\dagger_p a_q + \frac{1}{2} \sum_{pqrs} h_{pqrs} \, a^\dagger_p a^\dagger_q a_r a_s$ is an InteractionOperator

In [ ]:
# Set molecule parameters.
diatomic_bond_length = 4.
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
description = str(diatomic_bond_length)

# Set Hamiltonian parameters.
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData using Psi4.
molecule = MolecularData(geometry, basis, multiplicity,
                         description=description)
molecule = run_psi4(molecule, run_scf=True)

# Get the Hamiltonian in an active space.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    active_space_start, active_space_stop)

print molecular_hamiltonian

In [ ]:
fermion_hamiltonian = molecular_hamiltonian.get_fermion_operator()
qubit_hamiltonian = fermion_hamiltonian.bravyi_kitaev_transform()
print qubit_hamiltonian

In [ ]:
import scipy
import numpy

n_orbitals = molecular_hamiltonian.n_qubits // 2
n_variables = n_orbitals * (n_orbitals - 1) / 2
random_angles = numpy.pi * (1. - 2. * numpy.random.rand(n_variables))
kappa = numpy.zeros((n_orbitals, n_orbitals))

index = 0
for p in range(n_orbitals):
    for q in range(p + 1, n_orbitals):
        kappa[p, q] = random_angles[index]
        kappa[q, p] = -numpy.conjugate(random_angles[index])
        index += 1

rotation_matrix = scipy.linalg.expm(kappa)

In [ ]:
qubit_hamiltonian_1 = molecular_hamiltonian.jordan_wigner_transform()
sparse_hamiltonian_1 = molecular_hamiltonian.get_sparse_operator()
energy_1, state_1 = sparse_hamiltonian_1.get_ground_state()

molecular_hamiltonian.rotate_basis(rotation_matrix)

qubit_hamiltonian_2 = molecular_hamiltonian.jordan_wigner_transform()
sparse_hamiltonian_2 = qubit_hamiltonian.get_sparse_operator()
energy_2, state_2 = sparse_hamiltonian_2.get_ground_state()

print qubit_hamiltonian_1 - qubit_hamiltonian_2
print energy_1 - energy_2